GitHub  
https://github.com/clovaai/CRAFT-pytorch  
論文  
https://arxiv.org/abs/1904.01941    
  
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/CRAFT_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GitHubからCRAFTのソースコードを取得

In [ ]:
%cd /content/
!git clone https://github.com/clovaai/CRAFT-pytorch.git

## ライブラリのインストール

In [ ]:
%cd /content/CRAFT-pytorch/
!pip uninstall torch torchtext torchaudio -y
!pip install -r requirements.txt

## ライブラリのインポート

In [ ]:
import os
import gdown
import glob
from PIL import Image
import matplotlib.pyplot as plt
import random
import shutil

import torch

## 学習済みモデルのダウンロード

In [ ]:
%cd /content/CRAFT-pytorch/
!mkdir pretrained
%cd /content/CRAFT-pytorch/pretrained

for google_drive_file_id in ['1Jk4eGD7crsqCCg9C9VjCLkMN3ze8kutZ']:
  gdown.download(
      url='https://drive.google.com/uc?id={}'.format(google_drive_file_id),
      output="./craft_mlt_25k.pth", 
      quiet=False
      )

## データセットのダウンロード

In [ ]:
%cd /content/CRAFT-pytorch/
!mkdir dataset
%cd /content/CRAFT-pytorch/dataset

for google_drive_file_id in ['1sptDnAomQHFVZbjvnWt2uBvyeJ-gEl-A']:
  gdown.download(
      url='https://drive.google.com/uc?id={}'.format(google_drive_file_id),
      output="./icdar2013.zip", 
      quiet=False
      )

# ダウンロードしたzipファイルの解凍
# 解凍したファイルすべてが出力されることを避けるため > /dev/null で出力を捨てる
!unzip ./icdar2013.zip > /dev/null

# icdar2013 testファイルリストを取得
test_path = "/content/CRAFT-pytorch/dataset/icdar2013/test_images/"
test_path_list = glob.glob(test_path + "*.jpg")

## 文字検出(Character Detection)

In [ ]:
# 文字検出対象のファイル選定
target_path = "/content/CRAFT-pytorch/inputs/"
if os.path.exists(target_path):
  shutil.rmtree(target_path)
  os.makedirs(target_path, exist_ok=True)
else:
  os.makedirs(target_path, exist_ok=True)

# 文字検出用にランダムに5枚選択
target = random.sample(test_path_list, 5)

for file_path in target:
  shutil.copy(file_path, target_path)

target_path_list = glob.glob(target_path + "*.jpg")

# 文字検出対象画像の表示
for t_path in target_path_list:
  image = Image.open(t_path).convert("RGB")
  plt.figure(figsize=(6, 6))
  plt.imshow(image)

In [ ]:
%cd /content/CRAFT-pytorch/

# GPUが使用可能であればTrue、不可であればFalse
device = torch.cuda.is_available()
test_img = test_path_list[0]

!python3 test.py \
  --trained_model=/content/CRAFT-pytorch/pretrained/craft_mlt_25k.pth \
  --test_folder=$target_path \
  --cuda=$device

## 文字検出結果の表示

In [ ]:
results = glob.glob("/content/CRAFT-pytorch/result/*.jpg")
results = [f for f in results if not 'mask' in f ]

# 文字検出対象画像の表示
for result in results:
  image = Image.open(result).convert("RGB")
  plt.figure(figsize=(6, 6))
  plt.imshow(image)